This small script is preparing the Zika.DVG files to be merged into files used for input to the DVG script.

This takes the input of running the vardvg sh script on the subsampled bam files 

In [77]:
require('plyr')
library('ggplot2')
library('tidyverse')
library('reshape2')
library('grid')
require('gridExtra')
library('plotly')
library('glue')

In [78]:
wkdir='/home/kate/Lab/zika_files/ReAlignment/SubVariants/SubDVG_PCR/AllSub/'
setwd(wkdir)

In [79]:
# make a directory to save new files 
if (!dir.exists(glue("{wkdir}RenamedSubDVGFiles"))) {
      dir.create(glue("{wkdir}RenamedSubDVGFiles"))
    }

In [80]:
# input: split read data frame and frequency of subsampling
# output: split read dataframe but with frequency after pcr information and additional name info removed to run through di findder script
SubFreq = function(df, freq){
    
    df$name = gsub(glue(".zika.rmd.star.{freq}.sorted.bam.{freq}.split.txt"),glue("_{freq}"), df$name)
    
    return(df)
}



In [81]:
# write the altered split read file to csv output
# input dataframe, freq of subsampling, and save directory - where to save csv file 
# output csv file in save directory

WriteSplit = function(df, freq, savedir){

    write.csv(df, 
             file = glue("{savedir}/{freq}.Renamed.SplitReads.csv"),
             row.names = FALSE)
    
}



In [82]:
# write update features file to csv file
# input: dataframe, freq of subsampling, and savedirectory
# output: csv file with updated information 

WriteFeatures = function(df, freq, savedir) {
    
    # writing to table allows you to ignore column names 
    
    write.table(df, 
         file = glue("{savedir}/{freq}.Renamed.FeaturesCounts.csv"),
         row.names = FALSE,
         col.names = FALSE,
                sep = ',')
    
}



In [83]:
FeatRename = function(df, freq){ 
    df = df %>% 
      mutate_all(funs(str_replace(., "pcr1", glue("pcr1_{freq}"))))
    
    df = df %>% 
      mutate_all(funs(str_replace(., "pcr2", glue("pcr2_{freq}"))))
    
    df = df %>% 
      mutate_all(funs(str_replace(., "pcr3", glue("pcr3_{freq}"))))
    return(df)
}


In [84]:
#SplitReads first: 
# filenames
s1 = glue('{wkdir}1.SplitReads.csv')
s75 = glue('{wkdir}0.75.SplitReads.csv')
s50 = glue('{wkdir}0.50.SplitReads.csv')
s25 = glue('{wkdir}0.25.SplitReads.csv')
s10 = glue('{wkdir}0.10.SplitReads.csv')
s05 = glue('{wkdir}0.05.SplitReads.csv')

# read as csv files
sub1 = read.csv(file=s1,header=T,sep=",",na.strings = c('nan'))
sub75 = read.csv(file=s75,header=T,sep=",",na.strings = c('nan'))
sub50 = read.csv(file=s50,header=T,sep=",",na.strings = c('nan'))
sub25 = read.csv(file=s25,header=T,sep=",",na.strings = c('nan'))
sub10 = read.csv(file=s10,header=T,sep=",",na.strings = c('nan'))
sub05 = read.csv(file=s05,header=T,sep=",",na.strings = c('nan'))

In [85]:
# rename split read files and save
x1 = SubFreq(sub1, "1")
WriteSplit(x1, "1", glue("{wkdir}RenamedSubDVGFiles"))

x75 = SubFreq(sub75, "0.75")
WriteSplit(x75, "0.75", glue("{wkdir}RenamedSubDVGFiles"))

x50 = SubFreq(sub50, "0.50")
WriteSplit(x50, "0.50", glue("{wkdir}RenamedSubDVGFiles"))

x25 = SubFreq(sub25, "0.25")
WriteSplit(x25, "0.25", glue("{wkdir}RenamedSubDVGFiles"))

x10 = SubFreq(sub10, "0.10")
WriteSplit(x10, "0.10", glue("{wkdir}RenamedSubDVGFiles"))

x05 = SubFreq(sub05, "0.05")
WriteSplit(x05, "0.05", glue("{wkdir}RenamedSubDVGFiles"))


In [86]:
numbrows = nrow(x1) + nrow(x75) + nrow(x50) + nrow(x25) + nrow(x10) + nrow(x05)
print(numbrows)

[1] 377561


In [87]:
# rbind the split read files
splitreads = rbind(x1, x75)
splitreads = rbind(splitreads, x50)
splitreads = rbind(splitreads, x25)
splitreads = rbind(splitreads, x10)
splitreads = rbind(splitreads, x05)
dim(splitreads)

[1] 377561      7

In [88]:
write.csv(splitreads, 
         file = glue("{wkdir}ZIKA.SplitReads.csv"),
         row.names = FALSE)

In [89]:
# feature count files
f1 = glue('{wkdir}1.FeaturesOutput.csv')
f75 = glue('{wkdir}0.75.FeaturesOutput.csv')
f50 = glue('{wkdir}0.50.FeaturesOutput.csv')
f25 = glue('{wkdir}0.25.FeaturesOutput.csv')
f10 = glue('{wkdir}0.10.FeaturesOutput.csv')
f05 = glue('{wkdir}0.05.FeaturesOutput.csv')

feat1 = read.csv(file=f1,header=F,sep=",",na.strings = c('nan'))
feat75 = read.csv(file=f75,header=F,sep=",",na.strings = c('nan'))
feat50 = read.csv(file=f50,header=F,sep=",",na.strings = c('nan'))
feat25 = read.csv(file=f25,header=F,sep=",",na.strings = c('nan'))
feat10 = read.csv(file=f10,header=F,sep=",",na.strings = c('nan'))
feat05 = read.csv(file=f05,header=F,sep=",",na.strings = c('nan'))

In [90]:
# rename columns of the feature files 
ft1 = FeatRename(feat1, "1")
WriteFeatures(ft1, "1", glue("{wkdir}RenamedSubDVGFiles"))
dim(ft1)

ft75 = FeatRename(feat75, "0.75")
WriteFeatures(ft75, "0.75", glue("{wkdir}RenamedSubDVGFiles"))
dim(ft75)

ft50 = FeatRename(feat50, "0.50")
WriteFeatures(ft50, "0.50", glue("{wkdir}RenamedSubDVGFiles"))
dim(ft50)

ft25 = FeatRename(feat25, "0.25")
WriteFeatures(ft25, "0.25", glue("{wkdir}RenamedSubDVGFiles"))
dim(ft25)

ft10 = FeatRename(feat10, "0.10")
WriteFeatures(ft10, "0.10", glue("{wkdir}RenamedSubDVGFiles"))
dim(ft10)

ft05 = FeatRename(feat05, "0.05")
WriteFeatures(ft05, "0.05", glue("{wkdir}RenamedSubDVGFiles"))
dim(ft05)

[1]   2 247

[1]   2 247

[1]   2 247

[1]   2 247

[1]   2 247

[1]   2 247

In [91]:
numbcols = 247 * 6 - (6*5)
print(numbcols)

[1] 1452


In [92]:
# merge feature files 
featall = merge(ft1, ft75, by=c('V1','V2','V3','V4','V5','V6'), all = TRUE)
featall = merge(featall, ft50, by=c('V1','V2','V3','V4','V5','V6'), all = TRUE)
featall = merge(featall, ft25, by=c('V1','V2','V3','V4','V5','V6'), all = TRUE)
featall = merge(featall, ft10, by=c('V1','V2','V3','V4','V5','V6'), all = TRUE)
featall = merge(featall, ft05, by=c('V1','V2','V3','V4','V5','V6'), all = TRUE)
dim(featall)

Warning message in merge.data.frame(featall, ft25, by = c("V1", "V2", "V3", "V4", :
“column names ‘V7.x’, ‘V8.x’, ‘V9.x’, ‘V10.x’, ‘V11.x’, ‘V12.x’, ‘V13.x’, ‘V14.x’, ‘V15.x’, ‘V16.x’, ‘V17.x’, ‘V18.x’, ‘V19.x’, ‘V20.x’, ‘V21.x’, ‘V22.x’, ‘V23.x’, ‘V24.x’, ‘V25.x’, ‘V26.x’, ‘V27.x’, ‘V28.x’, ‘V29.x’, ‘V30.x’, ‘V31.x’, ‘V32.x’, ‘V33.x’, ‘V34.x’, ‘V35.x’, ‘V36.x’, ‘V37.x’, ‘V38.x’, ‘V39.x’, ‘V40.x’, ‘V41.x’, ‘V42.x’, ‘V43.x’, ‘V44.x’, ‘V45.x’, ‘V46.x’, ‘V47.x’, ‘V48.x’, ‘V49.x’, ‘V50.x’, ‘V51.x’, ‘V52.x’, ‘V53.x’, ‘V54.x’, ‘V55.x’, ‘V56.x’, ‘V57.x’, ‘V58.x’, ‘V59.x’, ‘V60.x’, ‘V61.x’, ‘V62.x’, ‘V63.x’, ‘V64.x’, ‘V65.x’, ‘V66.x’, ‘V67.x’, ‘V68.x’, ‘V69.x’, ‘V70.x’, ‘V71.x’, ‘V72.x’, ‘V73.x’, ‘V74.x’, ‘V75.x’, ‘V76.x’, ‘V77.x’, ‘V78.x’, ‘V79.x’, ‘V80.x’, ‘V81.x’, ‘V82.x’, ‘V83.x’, ‘V84.x’, ‘V85.x’, ‘V86.x’, ‘V87.x’, ‘V88.x’, ‘V89.x’, ‘V90.x’, ‘V91.x’, ‘V92.x’, ‘V93.x’, ‘V94.x’, ‘V95.x’, ‘V96.x’, ‘V97.x’, ‘V98.x’, ‘V99.x’, ‘V100.x’, ‘V101.x’, ‘V102.x’, ‘V103.x’, ‘V104.x’, ‘V105.x’, ‘V106.x’

Warning message in merge.data.frame(featall, ft05, by = c("V1", "V2", "V3", "V4", :
“column names ‘V7.x’, ‘V8.x’, ‘V9.x’, ‘V10.x’, ‘V11.x’, ‘V12.x’, ‘V13.x’, ‘V14.x’, ‘V15.x’, ‘V16.x’, ‘V17.x’, ‘V18.x’, ‘V19.x’, ‘V20.x’, ‘V21.x’, ‘V22.x’, ‘V23.x’, ‘V24.x’, ‘V25.x’, ‘V26.x’, ‘V27.x’, ‘V28.x’, ‘V29.x’, ‘V30.x’, ‘V31.x’, ‘V32.x’, ‘V33.x’, ‘V34.x’, ‘V35.x’, ‘V36.x’, ‘V37.x’, ‘V38.x’, ‘V39.x’, ‘V40.x’, ‘V41.x’, ‘V42.x’, ‘V43.x’, ‘V44.x’, ‘V45.x’, ‘V46.x’, ‘V47.x’, ‘V48.x’, ‘V49.x’, ‘V50.x’, ‘V51.x’, ‘V52.x’, ‘V53.x’, ‘V54.x’, ‘V55.x’, ‘V56.x’, ‘V57.x’, ‘V58.x’, ‘V59.x’, ‘V60.x’, ‘V61.x’, ‘V62.x’, ‘V63.x’, ‘V64.x’, ‘V65.x’, ‘V66.x’, ‘V67.x’, ‘V68.x’, ‘V69.x’, ‘V70.x’, ‘V71.x’, ‘V72.x’, ‘V73.x’, ‘V74.x’, ‘V75.x’, ‘V76.x’, ‘V77.x’, ‘V78.x’, ‘V79.x’, ‘V80.x’, ‘V81.x’, ‘V82.x’, ‘V83.x’, ‘V84.x’, ‘V85.x’, ‘V86.x’, ‘V87.x’, ‘V88.x’, ‘V89.x’, ‘V90.x’, ‘V91.x’, ‘V92.x’, ‘V93.x’, ‘V94.x’, ‘V95.x’, ‘V96.x’, ‘V97.x’, ‘V98.x’, ‘V99.x’, ‘V100.x’, ‘V101.x’, ‘V102.x’, ‘V103.x’, ‘V104.x’, ‘V105.x’, ‘V106.x’

[1]    2 1452

In [93]:
write.table(featall, 
         file = glue("{wkdir}ZIKA.FeaturesOutput.csv"), 
         row.names = FALSE,
         col.names = FALSE,
           sep = ',')

The two outputs can now be put through the DVG script FindDI_star_15.py

python3 FindDI_star_15.py --strain ZIKA --bam_path ../PCR_SubSortedBam/ --ref ../../reference/zika/MR766.ZIKA.CDS.fas --file ../../SubVariants/SubDVG_PCR/AllSub/ZIKA.SplitReads.csv --idx ../../SubVariants/SubDVG_PCR/AllSub/ZIKA.FeaturesOutput.csv --save_dir ../../SubVariants/SubDVG_PCR/AllSub --align_length 25 --total_mismatch  1


